In [1]:
#import requests

import numpy as np
import tensorflow as tf
tf.enable_v2_behavior()

from rcnn_sat import preprocess_image
from rcnn_sat.bl_net import bl_net

# Restore model and extract activations
This example shows how to extract activations from a pre-trained model. We'll use the BL model trained on eco-set.

## Building the model

First, we build the graph for the model using a placeholder for the input.

In [2]:
input_layer = tf.keras.layers.Input((128, 128, 3))
model = bl_net(input_layer, classes=565, cumulative_readout=True)

Instructions for updating:
Colocations handled automatically by placer.


All model functions take an input layer as the first argument and the number of classes as the second. Note for ecoset `classes=565` and for ImageNet `classes=1000`.

BL has an additional argument specifying whether or not to use a cumulative readout. By deafult `cumulative_readout=False`.

## Loading the weights

Download the pre-trained weights for the models from OSF (all weights are available from this [link](https://osf.io/mz9hw/)).

In [ ]:
r = requests.get('https://osf.io/ac4ng/download')
with open('bl_ecoset.h5', 'wb') as f:  
    f.write(r.content)

Load the weights into the model

In [3]:
model.load_weights('bl_ecoset.h5')

## Extract activations

To get the activation in Keras we need to know the name of the layer. The name of the output at each layer and time step has the format `ReLU_Layer_{layer_index}_Time_{time_index}`. So, the output of the first layer on the third time step is labelled `ReLU_Layer_0_Time_2` (note that zero-indexing is used).

We use this to create a function mapping from input to output that we call to get a numpy array of the layer activations.

In [4]:
get_layer_activation = tf.keras.backend.function(
    [model.input],
    [model.get_layer('ReLU_Layer_0_Time_2').output])

Now we'll get the activations based on a random image. We scale the image into the correct range for the pre-trained networks by passing the image through `preprocess_image`. The image must be in `uint8` format (values in the range `[0, 255]`).

In [5]:
random_img = np.random.randint(0, 256, (1, 128, 128, 3), dtype=np.uint8)
preprocessed_img = preprocess_image(random_img)
get_layer_activation(preprocessed_img)

[array([[[[0.0000000e+00, 0.0000000e+00, 2.0927539e-01, ...,
           0.0000000e+00, 2.3100780e-01, 1.1727836e+00],
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
           0.0000000e+00, 1.3024559e+00, 4.8215500e-01],
          [4.8849031e-01, 1.7477043e+00, 0.0000000e+00, ...,
           0.0000000e+00, 1.9862010e+00, 3.1804517e-03],
          ...,
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
           0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          [1.1324693e+00, 0.0000000e+00, 0.0000000e+00, ...,
           0.0000000e+00, 9.5403779e-01, 2.7408990e-01],
          [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
           0.0000000e+00, 0.0000000e+00, 5.9002304e-01]],
 
         [[0.0000000e+00, 0.0000000e+00, 1.4243525e+00, ...,
           0.0000000e+00, 0.0000000e+00, 3.9274761e-01],
          [3.6923146e-01, 1.3568745e+00, 0.0000000e+00, ...,
           0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
          [0.0000000e+00, 1.5459092e+0